In [62]:
from gensim.models.doc2vec import Doc2Vec
from gensim.models import Word2Vec
import pickle
import numpy as np
import pandas as pd

In [63]:
from gensim.parsing.preprocessing import preprocess_string
import re
import string
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import stem_text
from gensim.parsing.preprocessing import strip_numeric
def remove_ip(s):
    # Replace all ip adresses with '<ip>' tag
    ip_regexp = r"\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b"
    return re.sub(ip_regexp, '<ip>', s)
def remove_email(s):
    # Replace all email adresses with '<email>' tag
    email_regexp = r"([a-zA-Z0-9_\-\.]+)@([a-zA-Z0-9_\-\.]+)\.([a-zA-Z]{2,5})"
    return re.sub(email_regexp, '<email>', s)
def remove_mailto(s):
    # Replace all "<mailto:<email>>" with <email>. Email adresses should be replaced by remove_email first.
    return s.replace("<mailto:<email>>", "<email>")
def remove_url(s):
    # Replace all url's with '<url>' tag
    url_regexp = r"((http|ftp|https):\/\/)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)"
    s = re.sub(url_regexp, '<url>', s)
    # Sometimes url's are inside <> so we need to replace <<url>> with <url>
    return s.replace("<<url>>", "<url>")
def remove_punc(s, exceptions):
    # Remove all punctuation from string with exceptions in list exceptions
    remove = string.punctuation
    for exception in exceptions:
        remove = remove.replace(exception, "")
    # Create the pattern
    pattern = r"[{}]".format(remove)

    return re.sub(pattern, "", s)
def remove_custom_stopwords(s, stopwords):
    for stopword in stopwords:
        s = s.replace(stopword, "")
    return s
def lower_case(s):
    return s.lower()
def preprocess_sentence_fn(s):
    # Preprocess a single sentence to a list of tokens
    punc_exceptions = ['<', '>']
    custom_stopwords = ['dear', 'sincerely', 'thanks', 'yours', 'regards']
    filters = [lower_case,
               remove_ip,
               remove_email,
               remove_mailto,
               #remove_url,
               lambda x: remove_punc(x, punc_exceptions),
               remove_stopwords,
               lambda x: remove_custom_stopwords(x, custom_stopwords),
               strip_multiple_whitespaces,
               stem_text,
               strip_numeric]
    out = preprocess_string(s, filters=filters)
    return out
def preprocess_docs_fn(docs):
    # Apply preprocess_sentence_fn to a list of sentances (docs) to get a list of lists
    return [preprocess_sentence_fn(s) for s in docs]

In [55]:
#debug: print the vectors with 0 

for i in range(len(mean_ticket_ans)):
    zero = np.count_nonzero(mean_ticket_ans[i])
    if zero != 128: 
        print(i)
        
for i in range(len(mean_faq_ans)):
    zero = np.count_nonzero(mean_ticket_ans[i])
    if zero != 128: 
        print(i)

NameError: name 'mean_ticket_ans' is not defined

Need to remove empty strings due to preprocessing 

In [142]:
# Read in the data
ticket_dat = pd.read_csv('../../data/12-04-ticket_dat.csv')
faq_dat = pd.read_csv('../../data/12-04-faq_dat.csv')
# Replace the NaNs
ticket_dat.fillna('', inplace=True)
faq_dat.fillna('', inplace=True)

In [143]:
# FAQ question
faq_ques = list(faq_dat.question)
n_faq_ques = len(faq_ques)
# FAQ answer
faq_ans = list(faq_dat.answer_title + " " + faq_dat.answer)
n_faq_ans = len(faq_ans)
#ticket question
ticket_ques = list(ticket_dat.question)
n_ticket_ques = len(ticket_ques)
#ticket ans
ticket_ans = list(ticket_dat.answer)
n_ticket_ans = len(ticket_ans)

In [134]:
# Model assumption: same embedding for all
all_docs = faq_ques + faq_ans + ticket_ques + ticket_ans
# Model assumption: two different embeddings
all_ans = faq_ans + ticket_ans

# create a dictionary storing the cut points for the four datasets so we can re-split them after.
# use like all_docs[id_dict['faq_ques']] to get all faq questions.
id_dict = {
    'faq_ques': range(0, n_faq_ques),
    'faq_ans': range(n_faq_ques, n_faq_ques + n_faq_ans),
    'ticket_ques': range(n_faq_ques + n_faq_ans, n_faq_ques + n_faq_ans + n_ticket_ques),
    'ticket_ans': range(n_faq_ques + n_faq_ans + n_ticket_ques, n_faq_ques + n_faq_ans + n_ticket_ques + n_ticket_ans)
}
all_docs_sep = {
    'faq_ques': faq_ques,
    'faq_ans': faq_ans,
    'ticket_ques': ticket_ques,
    'ticket_ans': ticket_ans}

In [135]:
all_docs_prepro = preprocess_docs_fn(all_docs)

In [136]:
for i in range(len(all_docs_prepro)):
    if not all_docs_prepro[i]:
        print('empty docs {}'.format(i))

empty docs 14


In [137]:
#check if datasets contain empty strings
faq_ques_prepro = preprocess_docs_fn(faq_ques)
for i in range(len(faq_ques_prepro)):
    if not faq_ques_prepro[i]:
        print('faq question {}'.format(i))

faq question 14


In [141]:
faq_ques[14]

'None'

In [138]:
faq_ans_prepro = preprocess_docs_fn(faq_ans)
for i in range(len(faq_ans_prepro)):
    if not faq_ans_prepro[i]:
        print('faq answer {}'.format(i))

In [139]:
ticket_ques_prepro = preprocess_docs_fn(ticket_ques)
for i in range(len(ticket_ques_prepro)):
    if not ticket_ques_prepro[i]:
        print('ticket question {}'.format(i))

In [140]:
ticket_ans_prepro = preprocess_docs_fn(ticket_ans)
for i in range(len(ticket_ans_prepro)):
    if not ticket_ans_prepro[i]:
        print('ticket answer {}'.format(i))